<a href="https://colab.research.google.com/github/JorgeLuizFranco/CompetitiveProgramming/blob/master/Small_data_learning_emb_var_to_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=blue>Introduction</font>

---
> Version 1.2

This is the notebook of the *Small-data learning* project.

## Mount gdrive

In [1]:
from google.colab import drive
from google.colab import files
import os


# Handy function to report errors.

def fault(msg):
  raise ValueError(msg)

# Mount the google drive

#drive.mount('/gdrive/Shareddrives/Data_Embrapa/Planilhas')
drive.mount('/gdrive/')


Mounted at /gdrive/


New in this version:


* Toggle normalization on/off
* Toggle delta filter on/off
* Choose between NCD and Euclidian distance metrics
* Improved docx vizualization


In [2]:
cp "/gdrive/Shareddrives/Data_Embrapa/Planilhas/dados_FPS_2019_2021.xlsx" "/boot/"


In [3]:
cp "/gdrive/Shareddrives/Data_Embrapa/Planilhas/Siglas_dados_FPS_2019_2021.xlsx" "/boot/"

# <font color=blue>Setup</font>
> Please, configure the following options.

In [4]:
## CHO0SE THE DATASET
import pandas as pd
#objectives = ["'Output'",	"'Tmelt'",	"'Power'","'Lmelting'",	"'WATS'",	"'ViscousD'"]
#vars = ["'Screw'", "'Grooves'", "'RBS'", "'Dext'", "'D1'", "'D3'", "'Lfeed'", "'L1'", "'L2'", "'L3'",	"'Pitch'", "'Tfeed'",	"'Tbarrel'", "'N'", "'Model'", "'bN'", "'hN1'", "'hN2'", "'NN'", "'Lg1'", "'Lg2'", "'B'", "'Type'", "'Lrbs'", "'Lrbs-L'", "'Nb'", "'ERROR'"]
type_of_columns=""# "numerical"

# Datasets for the paper
#cp "/gdrive/Shareddrives/Data_Embrapa/Planilhas/dados_FPS_2019_2021.xlsx" "/boot/"
data_path_with_names = "/boot/dados_FPS_2019_2021.xlsx"
data_path_embrapa= pd.read_excel(data_path_with_names)
print(data_path_embrapa.columns)
# Define decision variables and objetives
columns=[]
objectives = ["'24'"]  # objetivos relacionados com sustentabilidade (Sandra)
objectives_not_used=["'14'","'15'","'21'","'22'"]
vars = []
#data_path_embrapa.columns
idx=0
special_chars="=!#$%^&*()/ +çãáêéíôóúÃÁÂÊÉÍÔÓÚ"
for var in data_path_embrapa.columns:
    up= var.upper()
    for c in special_chars:
      if c==' ':
        var= var.replace(c,'_')
      elif c=='ç':
        var= var.replace(c,'c')
      elif c in 'ãáâÃÁÂ':
        var=var.replace(c,'a')
      elif c in 'êéÊÉ':
        var=var.replace(c,'e')
      elif c in 'íÍ':
        var=var.replace(c,'i')
      elif c in 'ôóÔÓ':
        var=var.replace(c,'o')
      elif c in 'úÚ':
        var=var.replace(c,'u')
      else:
        var=var.replace(c,'')
    if('RECEITA' in up or 'LUCRO' in up):  # Indicados pela Sandra
        #objectives.append(str("'"+str(idx)+"'"))
        objectives_not_used.append(str("'"+str(idx)+"'"))
    else:
        vars.append(str("'"+str(idx)+"'"))
    columns.append(var)
    idx+=1
df_numeric=data_path_embrapa.copy()
columns_names=df_numeric.columns
for i in range(len(df_numeric.columns)):
  # transform columns to numbers
  df_numeric = df_numeric.rename({df_numeric.columns[i]:str(i)}, axis='columns')
df_numeric.to_excel("/boot/data.xlsx",sheet_name='Plan1',index=False) 
if type_of_columns=="numerical":
  data_path="/boot/data.xlsx"
else:
  #cp "/gdrive/Shareddrives/Data_Embrapa/Planilhas/Siglas_dados_FPS_2019_2021.xlsx" "/boot/" #Siglas_dados_FPS_2019_2021.xlsx
  data_path="/boot/Siglas_dados_FPS_2019_2021.xlsx"
  df_siglas=pd.read_excel(data_path)
  df_siglas.reset_index()
  for i in range(len(objectives)):
    objectives[i]=str("'"+df_siglas.columns[int(objectives[i].replace("'",""))]+"'")
  for i in range(len(vars)):
    vars[i]=str("'"+df_siglas.columns[int(vars[i].replace("'",""))]+"'")
print(vars+objectives)
#print(len(df_siglas.columns))
#df_siglas.columns
df_siglas

Index(['Nome_fazenda', 'Sub_região', 'Ano', 'Sistema', 'Tamanho (há)',
       'Perfil de gestão', 'Area_pastagem_cultivada (há)',
       'Proporção de formação de pastagem cultivada da área total',
       'Proporção de pastagem (nativa +exótica) da área',
       'Área (pastagem nativa total+exótica)',
       'Área de pastagem nativa total (há)',
       'Proporção de pastagem nativa total (%)', 'Grau de inundação_cheia (%)',
       'Área iundada (há)',
       'Proporção pasto nativo de melhor qualidade (área umida e sazonal)',
       'Area_pastagem nativa melhor qualidade (há)',
       'Taxa de lotação média (UA/há)',
       'Proporção de áreas florestadas natural (% da área total)',
       'Proporção de pastagens de qualidade inferior (savanas e campos sujos altos da área total)',
       'Area_pastagem_ qualidade inferior', 'Taxa de lotação média ',
       'Valorprodutivo_paisagens _VPUP_natural (relação)',
       'Valorprodutivo_paisagens _VPUP_alteradonatural (relação)',
       'Índi

,Nome,SubReg,Ano,Sist,Area,Perfil,PastCult,PC_At,PCPN_AT,AreaPCPN,...,RBGado,CFixoCabporM,CAni_arroba,CDiver_arroba,CFixo_arroba,Lucro_Bruto,RL,COE_RL,RLO_RB,LucroLiq_RB
0,Americana,Poconé,2019.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,285.54,12.23,43.40,23.59,52.15,97.79,278.26,0.657526,0.34,0.50
1,Americana,Poconé,2020.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,267.72,.,52.61,35.63,19.86,94.20,256.49,0.970000,0.03,0.35
2,Floresta,Cáceres,2019.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,956.00,7.56,44.84,18.75,20.36,360.28,930.19,0.660000,0.34,0.57
3,Floresta,Cáceres,2020.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,555.04,.,71.98,159.52,99.31,379.31,546.65,2.040000,-1.04,0.68
4,N.S. do Machadinho,Cáceres,2019.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,93.40,0.74,203.12,191.76,352.21,NaN,NaN,5.890000,-4.89,3.22
5,N.S. do Machadinho,Cáceres,2020.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,NaN,NaN,NaN,NaN,NaN,521.68,NaN,NaN,NaN,NaN
6,Bela Vista,Poconé,2019.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,1824,...,693.76,4,96.63,52.02,45.45,83.89,680.12,1.110000,-0.12,0.12
7,Bela Vista,Poconé,2020.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,.,...,55.61,.,217.67,444.21,447.71,24.47,52.62,0.690000,0.31,0.44
8,Conceição,Poconé,2019.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,175.91,1.47,6.22,39.58,30.48,122.88,172.07,0.420000,0.58,0.70
9,Conceição,Poconé,2020.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,218.20,.,187.22,24.31,196.30,95.77,214.46,0.630000,0.37,0.44


In [5]:
#from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


number of samples tree

In [6]:
##
## Chose resampling options
##

# If sample_total is 1, then resampling is disabled: cophenetic and membership
# information refers to the tree resulting from the original table. Otherwise
# resampling is enabled: cophenetic and membership information refer to the
# consensus tree formed by resamplings of the input data.

sample_total = 32 # 23       # If 1, then resampling is disabled
sample_replacement = True   # If resampling on, with or without replacement 


##
## DOCX report path
##

# The resulting report in docx format will be saved here

report_path = "/gdrive/Shareddrives/Data_Embrapa/Reports/report" +str(sample_total)+".docx"


In [7]:
##
## Transform input data (preprocessing)
##

# Apply normalization 

transf_normalize = False #True

# Use delta encoding

transf_delta = False

In [8]:
##
## Choose between NCD and Vector Distance
##

# Values = 'ncd', 'vd'

dist_type = 'ncd'

In [9]:
##
## Chose the colors used by colored trees
##

alt_colors = [
 'rgba(80%, 0%, 0%,1.000)',  # Red
 'rgba(0%, 80%, 0%,1.000)',  # Green
 'rgba(0%,0%,80%,1.000)',    # Blue
 'rgba(0%,40%,40%,1.000)',   # Cyan
 'rgba(80%,80%,0%,1.000)',   # Yellow
 'rgba(80%,0%,80%,1.000)',   # Magenta
 'rgba(100%,50%,0%,1.000)',  # Orange
 'rgba(50%,0%,100%,1.000)',  # Purple
 'rgba(100%,0%,100%,1.000)', # Pink
 'rgba(0%,50%,100%,1.000)',  # Sky-blue 
 'rgba(100%,20%,50%,1.000)', # Rose
 'rgba(50%,50%,0%,1.000)',   # Olive
 'rgba(40%,20%,0%,1.000)',   # Brown
 'rgba(40%,40%,40%,1.000)',  # Gray
 'rgba(0%,0%,0%,1.000)'      # Black
]

color_names = ['Red', 'Green', 'Blue', 'Cyan', 'Yellow', 'Magenta', 'Orange', 'Purple', 'Pink', 'Sky-blue', 'Rose', 'Olive', 'Brown', 'Gray', 'Black']

# A function to print colored text

def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

show_color_list = False         # Ucomment to show color list

if show_color_list == True:
  count=0;
  for i in alt_colors:
    s = i.replace('rgba','')
    s = s.replace('(','')
    s = s.replace(')','')
    r,g,b,a = s.split(',')
    r = r.split('%')[0]
    g = g.split('%')[0]
    b = b.split('%')[0]
    red = int(float(r)*255/100)
    green = int(float(g)*255/100)
    blue = int(float(b)*255/100)
    rgb_color = "{0:>3d} {1:>3d} {2:>3d}".format(red,green,blue)
    print(colored(0,0,0,"{0:2d}".format(count)), colored(red,green,blue,  rgb_color + '   ' +color_names[count]))
    count += 1;

## <font color=blue>Set up file system</font>
> <font color=darkblue>If execution stalls, check this cell to verify authentication</font>

## Clean up old files

In [10]:
import os
import shutil

files = os.listdir('/content')
files.remove('.config')


try:
  for i in files:
    pi = '/content/' + i
    if os.path.isdir(pi):
      shutil.rmtree(pi)
    elif os.path.isfile(pi):
      print()
      #os.remove(pi)
except OSError as e:
  print("Error: %s : %s" % (pi, e.strerror))



Error: /content/drive : Operation canceled


# <font color=blue>Read input</font>


In [11]:
from openpyxl import Workbook, load_workbook
import csv

wb = Workbook()
ws = wb.active

supported_extensions = ['.xls', '.xlsx', '.csv']

# Check if the data file exists

if not os.path.lexists(data_path):
  fault ("Data file "+data_path+" not found")

# Check if data file is of one of the supported types

file_name, file_extension = os.path.splitext(data_path)
if file_extension.lower() not in supported_extensions:
  fault()

# If data file is a CSV file, convert it to XSL and save a copy of the 
# converted file for convenience.

if file_extension.lower() == '.csv':
  print ("It's a csv, converting to xls.")
  with open(data_path, 'r') as f:
    for row in csv.reader(f, delimiter=';'):
      ws.append(row)
    wb.save(data_path.replace(file_extension,".xlsx"))

# Load the XSL file

wb = load_workbook(data_path)

In [12]:
# Get the spreadshit

sheets = wb.sheetnames
ws = wb[sheets[0]]

In [13]:
#@title
# Get a Pandas dataframe

import pandas as pd

table = pd.read_excel(data_path, header=[0])
df = pd.DataFrame(table)


In [14]:
df

,Nome,SubReg,Ano,Sist,Area,Perfil,PastCult,PC_At,PCPN_AT,AreaPCPN,...,RBGado,CFixoCabporM,CAni_arroba,CDiver_arroba,CFixo_arroba,Lucro_Bruto,RL,COE_RL,RLO_RB,LucroLiq_RB
0,Americana,Poconé,2019.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,285.54,12.23,43.40,23.59,52.15,97.79,278.26,0.657526,0.34,0.50
1,Americana,Poconé,2020.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,267.72,.,52.61,35.63,19.86,94.20,256.49,0.970000,0.03,0.35
2,Floresta,Cáceres,2019.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,956.00,7.56,44.84,18.75,20.36,360.28,930.19,0.660000,0.34,0.57
3,Floresta,Cáceres,2020.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,555.04,.,71.98,159.52,99.31,379.31,546.65,2.040000,-1.04,0.68
4,N.S. do Machadinho,Cáceres,2019.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,93.40,0.74,203.12,191.76,352.21,NaN,NaN,5.890000,-4.89,3.22
5,N.S. do Machadinho,Cáceres,2020.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,NaN,NaN,NaN,NaN,NaN,521.68,NaN,NaN,NaN,NaN
6,Bela Vista,Poconé,2019.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,1824,...,693.76,4,96.63,52.02,45.45,83.89,680.12,1.110000,-0.12,0.12
7,Bela Vista,Poconé,2020.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,.,...,55.61,.,217.67,444.21,447.71,24.47,52.62,0.690000,0.31,0.44
8,Conceição,Poconé,2019.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,175.91,1.47,6.22,39.58,30.48,122.88,172.07,0.420000,0.58,0.70
9,Conceição,Poconé,2020.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,218.20,.,187.22,24.31,196.30,95.77,214.46,0.630000,0.37,0.44


# <font color=blue>Transform input data</font>

### Normalize and add noise
> Avoid ill-conditioning due to zero distances <br/>
> Add small random noise

In [15]:
import numpy as np

if transf_normalize == True:
  df_norm = df.copy()

  #dft = df[:].astype(float).copy()
  for i in df.columns:
    if df[i].dtypes != np.object:
      df[i].astype(float)
  dft = df.copy()

  maxs = dft[:].max()
  for m, c in zip(maxs,df.columns):
    # a noise avoids zero distances, as consequence, their ill-conditioning due to them
    ##df[c] = dft[c].apply(lambda x: ((x/m)+np.random.normal(0,10**-12,1)[0]).round(6)  if m !=0 else 0+np.random.normal(0,10**-12,1)[0])
    if dft[c].dtype != np.object:
      dft[c] = dft[c].apply(lambda x: (x/m if m !=0 else x) )
    #dft[c] = dft[c].apply(lambda x: (x/m  if m !=0 else 0) )
    #translating data to the center of each variable range: (x - mu)
    #Each variable has the same center of mass, zero
    #df[c] = dft[c].apply(lambda x: (x - dft[c].mean() + 10**-6)) #+10**-6 to avoid zero weights between vars
    df_norm[c] = dft[c].copy()
  df_norm.columns = dft.columns
  df = df_norm.copy()

In [16]:
df.columns

Index(['Nome', 'SubReg', 'Ano', 'Sist', 'Area', 'Perfil', 'PastCult', 'PC_At',
       'PCPN_AT', 'AreaPCPN', 'AreaPN', 'ProporPN', 'GrauInund', 'AreaInund',
       'PropPN_US', 'AreaPN_US', 'TL_US', 'PropAF', 'PropPInf', 'AreaPInf',
       'TL_PI', 'VPUP_Nat', 'VPUP_Alt', 'IAPNP', 'ICDP', 'TL_PN', 'PC_Nece',
       'Valor_PN', 'NumCab', 'N_Vacas', 'Id_PriCria', 'IEP', 'TPrenhez',
       'TNat', 'TVacLac', 'nascido', 'N_Bezdes', 'UA_total_ponderado', 'Prod',
       'N_Bezdes_Area', 'T_Desfrute', 'TL_AreaTot', 'Perc_Vacas',
       'PerceRetNov', 'T_Descarte', 'TDfrute_TDc', 'T_Cres', 'CustoManejo',
       'CustoDiversos', 'CFixo_julho2019', 'COE', 'COT', 'CT', 'RBGado',
       'CFixoCabporM', 'CAni_arroba', 'CDiver_arroba', 'CFixo_arroba',
       'Lucro_Bruto', 'RL', 'COE_RL', 'RLO_RB', 'LucroLiq_RB'],
      dtype='object')

## Use delta encoding

In [17]:

delta_down = 0
delta_none = 1
delta_up = 2

if transf_delta == True:
  dft = df[:].astype(int).copy()

  for col in dft:
    last = dft.loc[0,col]  
    dft.loc[0,col] = delta_none
    for row in range(1,30):

      tmp = dft.loc[row,col]

      if dft.loc[row, col] > last:
        dft.loc[row,col]=delta_up
      elif dft.loc[row, col] < last:
        dft.loc[row,col]=delta_down
      else:
        dft.loc[row,col]=delta_none
      last = tmp
  df = dft.copy()



In [18]:
df

,Nome,SubReg,Ano,Sist,Area,Perfil,PastCult,PC_At,PCPN_AT,AreaPCPN,...,RBGado,CFixoCabporM,CAni_arroba,CDiver_arroba,CFixo_arroba,Lucro_Bruto,RL,COE_RL,RLO_RB,LucroLiq_RB
0,Americana,Poconé,2019.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,285.54,12.23,43.40,23.59,52.15,97.79,278.26,0.657526,0.34,0.50
1,Americana,Poconé,2020.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,267.72,.,52.61,35.63,19.86,94.20,256.49,0.970000,0.03,0.35
2,Floresta,Cáceres,2019.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,956.00,7.56,44.84,18.75,20.36,360.28,930.19,0.660000,0.34,0.57
3,Floresta,Cáceres,2020.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,555.04,.,71.98,159.52,99.31,379.31,546.65,2.040000,-1.04,0.68
4,N.S. do Machadinho,Cáceres,2019.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,93.40,0.74,203.12,191.76,352.21,NaN,NaN,5.890000,-4.89,3.22
5,N.S. do Machadinho,Cáceres,2020.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,NaN,NaN,NaN,NaN,NaN,521.68,NaN,NaN,NaN,NaN
6,Bela Vista,Poconé,2019.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,1824,...,693.76,4,96.63,52.02,45.45,83.89,680.12,1.110000,-0.12,0.12
7,Bela Vista,Poconé,2020.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,.,...,55.61,.,217.67,444.21,447.71,24.47,52.62,0.690000,0.31,0.44
8,Conceição,Poconé,2019.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,175.91,1.47,6.22,39.58,30.48,122.88,172.07,0.420000,0.58,0.70
9,Conceição,Poconé,2020.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,218.20,.,187.22,24.31,196.30,95.77,214.46,0.630000,0.37,0.44


*texto em itálico*## Avoid zero distances
This is needed for vector-distance metric

In [19]:
if dist_type == 'vd':
  df_noise = df.copy()
  dft = df[:].astype(float).copy()
  maxs = dft[:].max()
  for m, c in zip(maxs,df.columns):
    #dft[c] = dft[c].apply(lambda x: (x  if x !=0 else 0+10**-6) )
    #dft[c] = dft[c].apply(lambda x: (x  + 10**-6) )
    if dft[c].dtype != np.object:
      dft[c] = dft[c].apply(lambda x: (x  + np.random.uniform(0,1,1)[0]*10**-6 ))
    df_noise[c] = dft[c].copy()
  df_noise.columns = dft.columns
  df = df_noise.copy()
df

,Nome,SubReg,Ano,Sist,Area,Perfil,PastCult,PC_At,PCPN_AT,AreaPCPN,...,RBGado,CFixoCabporM,CAni_arroba,CDiver_arroba,CFixo_arroba,Lucro_Bruto,RL,COE_RL,RLO_RB,LucroLiq_RB
0,Americana,Poconé,2019.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,285.54,12.23,43.40,23.59,52.15,97.79,278.26,0.657526,0.34,0.50
1,Americana,Poconé,2020.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,267.72,.,52.61,35.63,19.86,94.20,256.49,0.970000,0.03,0.35
2,Floresta,Cáceres,2019.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,956.00,7.56,44.84,18.75,20.36,360.28,930.19,0.660000,0.34,0.57
3,Floresta,Cáceres,2020.0,Completo,13559.0,Estabelecido,3898.0,28.7,71.0,10758,...,555.04,.,71.98,159.52,99.31,379.31,546.65,2.040000,-1.04,0.68
4,N.S. do Machadinho,Cáceres,2019.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,93.40,0.74,203.12,191.76,352.21,NaN,NaN,5.890000,-4.89,3.22
5,N.S. do Machadinho,Cáceres,2020.0,Cria,3324.5,EM estabelecimento,395.5,11.9,92.4,3101,...,NaN,NaN,NaN,NaN,NaN,521.68,NaN,NaN,NaN,NaN
6,Bela Vista,Poconé,2019.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,1824,...,693.76,4,96.63,52.02,45.45,83.89,680.12,1.110000,-0.12,0.12
7,Bela Vista,Poconé,2020.0,Cria-recria,20406.0,Estabelecido,730.0,3.6,83.3,.,...,55.61,.,217.67,444.21,447.71,24.47,52.62,0.690000,0.31,0.44
8,Conceição,Poconé,2019.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,175.91,1.47,6.22,39.58,30.48,122.88,172.07,0.420000,0.58,0.70
9,Conceição,Poconé,2020.0,Cria,9001.0,Estabelecido,1000.0,11.1,73.0,6841,...,218.20,.,187.22,24.31,196.30,95.77,214.46,0.630000,0.37,0.44


# <font color=blue>Resample</font>
> Resample the rows in the worksheet 

In [20]:
# Get a list of resampled dataframes

resampled_df_l = [df]             # The original data set
for i in range(sample_total-1):   # Plus a few more
  resampled_df_l += [df.sample(n=df.shape[0], replace=sample_replacement)]

In [21]:
 resampled_df_l[1]

,Nome,SubReg,Ano,Sist,Area,Perfil,PastCult,PC_At,PCPN_AT,AreaPCPN,...,RBGado,CFixoCabporM,CAni_arroba,CDiver_arroba,CFixo_arroba,Lucro_Bruto,RL,COE_RL,RLO_RB,LucroLiq_RB
21,Fartura,Barao do Melgaço,NaN,Cria-recria,8000.0,Estabelecido,536.0,6.7,75.4,6164,...,671.70,3.95,17.45,10.63,67.33,386.94,661.62,0.510000,0.49,0.58
1,Americana,Poconé,2020.0,Cria_recria,1001.0,Estabelecido,698.0,69.7,81.0,830,...,267.72,.,52.61,35.63,19.86,94.20,256.49,0.970000,0.03,0.35
20,Fartura,Barao do Melgaço,NaN,Cria-recria,8000.0,Estabelecido,536.0,6.7,75.4,6164,...,853.73,5.53,26.17,31.16,36.77,482.25,834.06,0.610000,0.39,0.56
22,Toca do Nelore,Barao do Melgaço,NaN,Cria,2508.0,Estabelecido,1010.0,40.3,73.4,2110,...,392.39,2.17,79.89,28.24,86.20,70.77,384.89,1.070000,-0.09,0.18
12,Recreio,Cáceres,NaN,Cria,14428.0,Estabelecido,300.0,2.0,92.8,13411,...,919.76,9.94,28.14,5.26,6.94,560.95,905.97,0.450000,0.56,0.61
20,Fartura,Barao do Melgaço,NaN,Cria-recria,8000.0,Estabelecido,536.0,6.7,75.4,6164,...,853.73,5.53,26.17,31.16,36.77,482.25,834.06,0.610000,0.39,0.56
21,Fartura,Barao do Melgaço,NaN,Cria-recria,8000.0,Estabelecido,536.0,6.7,75.4,6164,...,671.70,3.95,17.45,10.63,67.33,386.94,661.62,0.510000,0.49,0.58
20,Fartura,Barao do Melgaço,NaN,Cria-recria,8000.0,Estabelecido,536.0,6.7,75.4,6164,...,853.73,5.53,26.17,31.16,36.77,482.25,834.06,0.610000,0.39,0.56
19,Faoreste,Barao do Melgaço,NaN,Cria-recria,5450.0,Estabelecido,3000.0,57.7,95.8,5108,...,678.57,3.6,49.57,15.18,53.95,668.39,602.57,0.630000,0.37,0.45
24,São Benedito,Barao do Melgaço,NaN,Cria,4453.0,Estabelecido,2800.0,62.9,71.6,3984,...,1106.46,6.53,38.24,23.97,43.46,475.05,1083.87,0.620000,0.38,0.43


In [22]:
# For each resampling, create a directory and save all column-files in it.

from distutils.dir_util import mkpath

resampledsource = os.path.join('/boot/', ws.title)
mkpath(resampledsource)
for i, dfrs in enumerate(resampled_df_l): 
  datasource = os.path.join(resampledsource, ws.title+str(i))
  mkpath(datasource)
  for v in dfrs.columns:
    if(type(v)==str):
      fname = v.replace("/","-")
      with open(os.path.join(datasource,fname), "w") as f: 
        f.write( str(dfrs[v]) ) 

# Show contents, for convenience
#files.view('/content/')

# <font color=blue>Build alternative distance matrix </font>
> Use scikit

In [23]:
#for i in df.columns:
#  if df[i].dtypes == np.object:
#    print ('Colum', i , 'is not numeric')
#    print ('This condition is not yet supported by the selected distance metric')
#    raise SystemExit("Aborted. Unsupported feature")

In [24]:
from sklearn.metrics import pairwise_distances 

def save_dist_matrix(dm, fname):
  f = open(fname, 'w+') #'dm-sklearn.csv', 'w+')
  f.write('x,y,zx,zy,zxy,ncd')
  leaves = df.columns
  i = 0
  for di, labels in zip(dm,df.columns):
    i += 1
    for j, dij in enumerate(di):
      #print(i, di, j)
      if j>i:
        #f.write('\nf'+str(i)+',f'+str(j)+',0,0,0,'+str(dij))
        f.write('\n'+str(leaves[i-1])+','+str(leaves[j])+',0,0,0,'+str(dij))
  f.write('\n')
  f.close()
  return 

os.makedirs('/content/input-dm-sklearn', exist_ok=True)

In [25]:
if dist_type == 'vd':
 for i, df_i in enumerate(resampled_df_l):
   Z = df_i.values.T
   dm = pairwise_distances(Z, Z, metric="euclidean")#metric="chebyshev")#, metric="nan_euclidean)#<==RESULTsimilar2CHEBYSHEV metric="seuclidean")#<==RESULTsimilar2CHEBYSHEV#, metric="minkowski")#<==RESULTsimilar2CHEBYSHEV#, metric="chebyshev")<==BestFound #, metric="manhattan") #, metric="cosine")#, metric="euclidean")
   dm = dm/np.max(dm)
   fname = '/content/input-dm-sklearn/dm-sklearn-'+str(i)+'.csv'
   save_dist_matrix(dm, fname)

# <font color=blue>Run DAMICORE</font>


## Install tools

> damicore, igraph, cairocffi



In [26]:
# Intall damicore
!pip install git+https://gitlab.uspdigital.usp.br/acbd/damicore-python3.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://gitlab.uspdigital.usp.br/acbd/damicore-python3.git to /tmp/pip-req-build-r8xicrja
  Running command git clone --filter=blob:none --quiet https://gitlab.uspdigital.usp.br/acbd/damicore-python3.git /tmp/pip-req-build-r8xicrja
  Resolved https://gitlab.uspdigital.usp.br/acbd/damicore-python3.git to commit 7a4a51525db0d698c588eecf24cd84d10473d46f
  Preparing metadata (setup.py) ... done
  Created wheel for damicore: filename=damicore-0.1.0-py3-none-any.whl size=49750 sha256=959b2595a5c965f0ab651751eec97e8c9e8b759cd0b99101af73eab7c69570e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4h7n1iy/wheels/ef/4c/a1/8d1482acc4802e689dc1c260619243a281a08cb271bdc2d7aa
Successfully built damicore


In [27]:
# Install igraph
!pip install python-igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.3 MB/s eta 0:00:00
  Created wheel for python-igraph: filename=python_igraph-0.10.4-py3-none-any.whl size=9073 sha256=5f1a4553141d32ce7eb10046b8be147e81a4a8b3fe58ab4a439ed34d9d92bac3
  Stored in directory: /root/.cache/pip/wheels/dc/07/ac/bff79052fd6222d1239b228cd24a47222f227c2350f9c4df01
Successfully built python-igraph


In [28]:
# Install cairocffi (run igraph utilities)
!pip install cairocffi #to run igraph drawing utilities 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cairocffi: filename=cairocffi-1.5.0-py3-none-any.whl size=90538 sha256=3c40ee3790740e8fc22e4f02d6707a4ad7e087b51e05723dd9f5c10fbb0ec552
  Stored in directory: /root/.cache/pip/wheels/34/c0/21/58ab88c0e5a36a32f10f5257afbd6e5fae805171da13691863
Successfully built cairocffi


In [29]:
!pip list -v | grep damicore > whereisit

## Actually execute damicore

In [30]:
# Locate damicore
!pip list -v | grep damicore > whereisit
f = open("whereisit")
strs = f.read()
dmpath = strs.split()[2]
dmpath

os.chdir(dmpath+"/damicore")

In [31]:
# Run DAMICORE for each resampling

import damicore   #%pycat damicore.py

results = "/content/results/"
mkpath(results)

for m in os.listdir(resampledsource):
  resampleddatasource = (os.path.join(resampledsource,m))  
  argv = ["damicore.py", "--compressor", "gzip", "--results-dir", results, resampleddatasource]
  args = argv[1:]
  print(args)
  print(columns)
  damicore.main(args)

['--compressor', 'gzip', '--results-dir', '/content/results/', '/boot/Plan1/Plan12']
['Nome_fazenda', 'Sub_regiao', 'Ano', 'Sistema', 'Tamanho_ha', 'Perfil_de_gestao', 'Area_pastagem_cultivada_ha', 'Proporcao_de_formacao_de_pastagem_cultivada_da_area_total', 'Proporcao_de_pastagem_nativa_exotica_da_area', 'area_pastagem_nativa_totalexotica', 'area_de_pastagem_nativa_total_ha', 'Proporcao_de_pastagem_nativa_total_', 'Grau_de_inundacao_cheia_', 'area_iundada_ha', 'Proporcao_pasto_nativo_de_melhor_qualidade_area_umida_e_sazonal', 'Area_pastagem_nativa_melhor_qualidade_ha', 'Taxa_de_lotacao_media_UAha', 'Proporcao_de_areas_florestadas_natural__da_area_total', 'Proporcao_de_pastagens_de_qualidade_inferior_savanas_e_campos_sujos_altos_da_area_total', 'Area_pastagem__qualidade_inferior', 'Taxa_de_lotacao_media_', 'Valorprodutivo_paisagens__VPUP_natural_relacao', 'Valorprodutivo_paisagens__VPUP_alteradonatural_relacao', 'indice_IAPNP', 'indice_de_conservacao_da_paisagem_ICDP', 'Taxa_de_lotacao

Performing NCD distance matrix calculation...
Compressing individual sources...
Compressing source pairs...
Simplifying graph...
Clustering elements...


Ano,1
Area,2
AreaInund,2
AreaPCPN,6
AreaPInf,6
AreaPN,6
AreaPN_US,6
CAni_arroba,4
CDiver_arroba,4
CFixoCabporM,5
CFixo_arroba,4
CFixo_julho2019,3
COE,2
COE_RL,0
COT,2
CT,2
CustoDiversos,3
CustoManejo,3
GrauInund,9
IAPNP,9
ICDP,0
IEP,9
Id_PriCria,1
LucroLiq_RB,8
Lucro_Bruto,3
N_Bezdes,1
N_Bezdes_Area,8
N_Vacas,1
Nome,7
NumCab,5
PCPN_AT,9
PC_At,9
PC_Nece,5
PastCult,0
Perc_Vacas,5
PerceRetNov,5
Perfil,7
Prod,5
PropAF,9
PropPInf,9
PropPN_US,9
ProporPN,1
RBGado,2
RL,2
RLO_RB,8
Sist,7
SubReg,7
TDfrute_TDc,5
TL_AreaTot,8
TL_PI,5
TL_PN,8
TL_US,5
TNat,0
TPrenhez,0
TVacLac,0
T_Cres,0
T_Descarte,0
T_Desfrute,5
UA_total_ponderado,2
VPUP_Alt,8
VPUP_Nat,8
Valor_PN,6
nascido,5
['--compressor', 'gzip', '--results-dir', '/content/results/', '/boot/Plan1/Plan110']
['Nome_fazenda', 'Sub_regiao', 'Ano', 'Sistema', 'Tamanho_ha', 'Perfil_de_gestao', 'Area_pastagem_cultivada_ha', 'Proporcao_de_formacao_de_pastagem_cultivada_da_area_total', 'Proporcao_de_pastagem_nativa_exotica_da_area', 'area_pastagem_nativa

Performing NCD distance matrix calculation...
Compressing individual sources...
Compressing source pairs...


Ano,0
Area,1
AreaInund,1
AreaPCPN,5
AreaPInf,5
AreaPN,5
AreaPN_US,6
CAni_arroba,3
CDiver_arroba,3
CFixoCabporM,5
CFixo_arroba,3
CFixo_julho2019,3
COE,3
COE_RL,2
COT,3
CT,3
CustoDiversos,3
CustoManejo,3
GrauInund,0
IAPNP,9
ICDP,1
IEP,9
Id_PriCria,4
LucroLiq_RB,8
Lucro_Bruto,3
N_Bezdes,4
N_Bezdes_Area,8
N_Vacas,4
Nome,7
NumCab,4
PCPN_AT,0
PC_At,0
PC_Nece,4
PastCult,1
Perc_Vacas,4
PerceRetNov,4
Perfil,7
Prod,6
PropAF,9
PropPInf,9
PropPN_US,9
ProporPN,4
RBGado,2
RL,2
RLO_RB,8
Sist,7
SubReg,7
TDfrute_TDc,6
TL_AreaTot,8
TL_PI,6
TL_PN,4
TL_US,6
TNat,1
TPrenhez,0
TVacLac,1
T_Cres,1
T_Descarte,1
T_Desfrute,6
UA_total_ponderado,3
VPUP_Alt,8
VPUP_Nat,8
Valor_PN,4
nascido,4
['--compressor', 'gzip', '--results-dir', '/content/results/', '/boot/Plan1/Plan118']
['Nome_fazenda', 'Sub_regiao', 'Ano', 'Sistema', 'Tamanho_ha', 'Perfil_de_gestao', 'Area_pastagem_cultivada_ha', 'Proporcao_de_formacao_de_pastagem_cultivada_da_area_total', 'Proporcao_de_pastagem_nativa_exotica_da_area', 'area_pastagem_nativa

...

Simplifying graph...
Clustering elements...
Performing NCD distance matrix calculation...
Compressing individual sources...
Compressing source pairs...


Ano,1
Area,6
AreaInund,6
AreaPCPN,3
AreaPInf,3
AreaPN,3
AreaPN_US,3
CAni_arroba,8
CDiver_arroba,8
CFixoCabporM,2
CFixo_arroba,8
CFixo_julho2019,8
COE,7
COE_RL,5
COT,7
CT,7
CustoDiversos,8
CustoManejo,8
GrauInund,0
IAPNP,0
ICDP,5
IEP,0
Id_PriCria,1
LucroLiq_RB,10
Lucro_Bruto,8
N_Bezdes,1
N_Bezdes_Area,10
N_Vacas,1
Nome,4
NumCab,1
PCPN_AT,0
PC_At,0
PC_Nece,2
PastCult,6
Perc_Vacas,2
PerceRetNov,1
Perfil,4
Prod,2
PropAF,0
PropPInf,0
PropPN_US,0
ProporPN,1
RBGado,6
RL,6
RLO_RB,10
Sist,4
SubReg,4
TDfrute_TDc,2
TL_AreaTot,10
TL_PI,3
TL_PN,10
TL_US,3
TNat,9
TPrenhez,9
TVacLac,5
T_Cres,9
T_Descarte,9
T_Desfrute,2
UA_total_ponderado,6
VPUP_Alt,10
VPUP_Nat,10
Valor_PN,2
nascido,1
['--compressor', 'gzip', '--results-dir', '/content/results/', '/boot/Plan1/Plan17']
['Nome_fazenda', 'Sub_regiao', 'Ano', 'Sistema', 'Tamanho_ha', 'Perfil_de_gestao', 'Area_pastagem_cultivada_ha', 'Proporcao_de_formacao_de_pastagem_cultivada_da_area_total', 'Proporcao_de_pastagem_nativa_exotica_da_area', 'area_pastagem_

Simplifying graph...
Clustering elements...
Performing NCD distance matrix calculation...
Compressing individual sources...
Compressing source pairs...


Ano,0
Area,5
AreaInund,6
AreaPCPN,2
AreaPInf,2
AreaPN,2
AreaPN_US,3
CAni_arroba,7
CDiver_arroba,7
CFixoCabporM,2
CFixo_arroba,7
CFixo_julho2019,7
COE,6
COE_RL,8
COT,6
CT,6
CustoDiversos,7
CustoManejo,7
GrauInund,0
IAPNP,0
ICDP,5
IEP,0
Id_PriCria,1
LucroLiq_RB,8
Lucro_Bruto,7
N_Bezdes,1
N_Bezdes_Area,9
N_Vacas,1
Nome,4
NumCab,2
PCPN_AT,0
PC_At,0
PC_Nece,2
PastCult,5
Perc_Vacas,2
PerceRetNov,2
Perfil,4
Prod,3
PropAF,0
PropPInf,0
PropPN_US,0
ProporPN,1
RBGado,6
RL,6
RLO_RB,8
Sist,4
SubReg,4
TDfrute_TDc,4
TL_AreaTot,9
TL_PI,3
TL_PN,2
TL_US,3
TNat,5
TPrenhez,5
TVacLac,5
T_Cres,5
T_Descarte,5
T_Desfrute,4
UA_total_ponderado,6
VPUP_Alt,9
VPUP_Nat,9
Valor_PN,2
nascido,2


Simplifying graph...
Clustering elements...


# Use matrix from sklearn metric

In [32]:
if dist_type == 'vd':
  for i in range(len(resampled_df_l)):
    print("\n",i) #--normalize-matrix
    argv = [  "damicore.py", "--normalize-weights", 
          "--matrix-mode", "/content/input-dm-sklearn/dm-sklearn-"+str(i)+".csv", 
          "--tree-output", "dm-sklearn-"+str(i)+".newick", 
          "--results-dir", "/content/results-dm-sklearn" ]
    args = argv[1:]
    damicore.main(args)

# <font color=blue>Phylogram analysis</font>

## Read newicks

In [33]:
dist_type

'ncd'

In [34]:
# Read all newick files (trees generated by DAMICORE) and form a single string.

if dist_type == 'ncd':
  res = '/content/results'
else:
  res = '/content/results-dm-sklearn'

plain_newicks = []
for tf in os.listdir(res):
  tf = tf+'/001-tree.newick'
  with open(os.path.join(res,tf), "r+") as f:
    plain_newicks += [ f.read() ]

# Format the all-newicks string so that each individual tree is terminated
# by a newline character.

string = ""
number_of_trees=0
for pn in plain_newicks:
  number_of_trees+=1
  string += pn+'\n' 

customorder = objectives + vars
print('number of trees: ',number_of_trees)
print ('We have', len(vars), ' decision variables and', len(objectives), 'objectives')

number of trees:  32
We have 57  decision variables and 1 objectives


## Install tools
> toytree

In [35]:
pip install git+https://github.com/eaton-lab/toytree.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eaton-lab/toytree.git to /tmp/pip-req-build-hrdpw8gh
  Running command git clone --filter=blob:none --quiet https://github.com/eaton-lab/toytree.git /tmp/pip-req-build-hrdpw8gh
  Resolved https://github.com/eaton-lab/toytree.git to commit 9d706e7caad63a17f2a8a6b97e752bc79c5eefed
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.8 MB/s eta 0:00:00
  Created wheel for toytree: filename=toytree-2.0.5-py3-none-any.whl size=115130 sha256=7b097dd6b426e7c07a59079

## Get an mtree
> mtree is a toytree object to handle multiple tress

In [36]:
import toytree as toytree # a tree plotting library
import toyplot            # a general plotting library
import numpy as np        # numerical library

# Load the forest as an mtree object.

mtree = toytree.mtree(string, tree_format=0)
print(len(mtree.treelist))
print(string)

32
((((('Ano':0.6624869347030615,('Area':0.6625594184143834,('AreaInund':0.6678508204888152,((((('CAni_arroba':0.6442400883032162,('CDiver_arroba':0.6365932653306912,'CFixo_arroba':0.6370385754653287):0.010759911696783864):0.03975635539755834,('CFixo_julho2019':0.6906744488462371,('CustoDiversos':0.6839511257979074,'CustoManejo':0.6855070022809104):0.007774057207812046):0.013459133957379577):0.0015861429717719533,'Lucro_Bruto':0.6955235562188455):0.015359428428611134,(('COE':0.6391199567690398,'COT':0.6769422193967634):0.011355828375622767,'CT':0.6868375537555832):0.01585612904040775):0.014302462358573697,(('RBGado':0.6523803030881734,'RL':0.6156609340252286):0.06599566977350674,'UA_total_ponderado':0.7449597882248162):0.00989854521904009):0.004600594292200277):0.014625020621597318):0.020369326300818946):0.005701987402334274,((((('AreaPCPN':0.5420645034555773,'AreaPN':0.5498895195329285):0.06375162523734651,'Valor_PN':0.6006892756994682):0.02361244199966295,('AreaPInf':0.63167077411805

## Cloud tree

In [37]:
#Uncomment to draw the cloud tree

canvas, axes, mark = mtree.draw_cloud_tree(
  layout = 'u',
   width = 800,
   height = 600,
   edge_style={
       "stroke-opacity": 0.1,
       "stroke-width": 3,
   },
);

for i in range(len(columns)):
  print(str(i),':',columns[i])

0 : Nome_fazenda
1 : Sub_regiao
2 : Ano
3 : Sistema
4 : Tamanho_ha
5 : Perfil_de_gestao
6 : Area_pastagem_cultivada_ha
7 : Proporcao_de_formacao_de_pastagem_cultivada_da_area_total
8 : Proporcao_de_pastagem_nativa_exotica_da_area
9 : area_pastagem_nativa_totalexotica
10 : area_de_pastagem_nativa_total_ha
11 : Proporcao_de_pastagem_nativa_total_
12 : Grau_de_inundacao_cheia_
13 : area_iundada_ha
14 : Proporcao_pasto_nativo_de_melhor_qualidade_area_umida_e_sazonal
15 : Area_pastagem_nativa_melhor_qualidade_ha
16 : Taxa_de_lotacao_media_UAha
17 : Proporcao_de_areas_florestadas_natural__da_area_total
18 : Proporcao_de_pastagens_de_qualidade_inferior_savanas_e_campos_sujos_altos_da_area_total
19 : Area_pastagem__qualidade_inferior
20 : Taxa_de_lotacao_media_
21 : Valorprodutivo_paisagens__VPUP_natural_relacao
22 : Valorprodutivo_paisagens__VPUP_alteradonatural_relacao
23 : indice_IAPNP
24 : indice_de_conservacao_da_paisagem_ICDP
25 : Taxa_de_lotacao_pasto_nativo
26 : Pastagem_cultivada_nece

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="600.0px" viewBox="0 0 800.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tad6507b422564a2cb6e5a4ea452d601f"> 'Sist' 'Perfil' 'SubReg' 'Nome' 'PerceRetNov' 'NumCab' 'nascido' 'Perc_Vacas' 'PC_Nece' 'CFixoCabporM' 'TL_US' 'TL_PI' 'Prod' 'T_Desfrute' 'TDfrute_TDc' 'AreaPN' 'AreaPCPN' 'Valor_PN' 'AreaPN_US' 'AreaPInf' 'N_Vacas' 'N_Bezdes' 'ProporPN' 'Id_PriCria' 'CustoManejo' 'CustoDiversos' 'CFixo_julho2019' 'CFixo_arroba' 'CDiver_arroba' 'CAni_arroba' 'Lucro_Bruto' 'COT' 'COE' 'CT' 'RL' 'RBGado' 'UA_total_ponderado' 'AreaInund' 'Area' 'Ano' 'VPUP_Nat' 'VPUP_Alt' 'TL_PN' 'TL_AreaTot' 'N_Bezdes_Area' 'RLO_RB' 'LucroLiq_RB' 'COE_RL' 'T_Cres' 'TPrenhez' 'T_Descarte' 'TVacLac' 'TNat' 'IAPNP' 'GrauInund' 'PCPN_AT' 'PC_At' 'IEP' 'PropPN_US' 'PropPInf' 'PropAF' 'ICDP' 'PastCult' <path d=

## Polytomy-resolved tree

In [38]:
# Uncomment to draw the polytomy-resolved tree

#tree = toytree.tree(string).resolve_polytomy(dist=.1).unroot()
#canvas, axes, mark = tree.draw(height=400,width=800, ts="s", layout="u", tip_labels_align=False)#, fixed_order=customorder)

## Consensus tree
> The tree is rooted at Output (currently hardcoaded)

In [39]:
ctre = mtree.get_consensus_tree().root("'Ano'")

# Uncomment to draw the consensus tree

#ctre.draw(ts='s', node_labels='support', **mystyle);

# Uncomment to print the consensus tree's newick

#ctre.newick

# <font color=blue> Analysis using raw data with phylograms </font>

### Value distributions

In [40]:
#m = []
#for c in range(1,ws.max_column+1):
#  v = []
#  for r in range(1,ws.max_row+1):
#    v += [ws.cell(row=r, column=c).value]
#  #m += [v[2:]]
#  m += [v[0:]]

In [41]:
#import numpy as np

#m_raw_values = np.array(m)
#m_raw_values

In [42]:
#distribution = dict()
#for r in m_raw_values:
#  try:
#    a = np.array((r[1:]).astype(np.float))
#    anorm = a/a.max()
#    distribution[r[0]] = anorm
#  except:
    #pass #a array may have literals as "values"
#    distribution[r[0]] = np.zeros(len(r[1:]))

In [43]:
#msize = 0
#for k, v in distribution.items():
#  #print(k, v.size)
#  if msize < v.size:
#    msize = v.size
#msize

### Cophenetic measures

In [44]:
# If the configuration variable sample_tree is grater than 1, then cophenetic 
# measurements are carried out for the consensus tree; otherwise, the
# cophenetic measurements are referring to the original tree.

if (sample_total > 1 ):
  the_tree = mtree.treelist[0]
else:
  the_tree = ctre


In [45]:
#for i, t in enumerate(mtree.treelist):
#  print(i,t)
#
#print (ctre)

#### Install ETE3

In [46]:
pip install ete3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.2-py3-none-any.whl size=2273011 sha256=cd99bdeea1dfbcc4635bd0f990078ff0eeafda8d792f1f1814098ac063a73945
  Stored in directory: /root/.cache/pip/wheels/db/69/ff/02a4fbe477131e200de6199e90ba7f327e57fbfc07203cd69e
Successfully built ete3


In [47]:
from ete3 import Tree

mTree = []
mTree = [ Tree(the_tree.newick) ]

#### Topology distances from a tree


In [48]:
from itertools import combinations, combinations_with_replacement

targetcomb = [ (t1.name, t2.name) for t1, t2 in combinations_with_replacement(mTree[0].get_leaves(),2) ]

In [49]:
labels = [ l.name for l in mTree[0].get_leaves() ]

topology_dist = dict()


for t1, t2 in targetcomb:
  if t1 == t2:
    d = 0
  else: 
    d = mTree[0].get_distance(target=t1, target2=t2, topology_only=True) #constrained to tree topology
  try: 
    topology_dist[t1] += [ d ]
  except:
    topology_dist[t1] =  [ d ] 
  if t1 != t2:
    try: 
      topology_dist[t2] += [ d ]
    except:
      topology_dist[t2] = [ d ]

In [50]:
labels = topology_dist.keys()
result = topology_dist.values()
data = list(result)
np_topology_dist = np.array(data)
print(labels)

dict_keys(["'PastCult'", "'ICDP'", "'PropAF'", "'PropPInf'", "'PropPN_US'", "'IEP'", "'PC_At'", "'PCPN_AT'", "'GrauInund'", "'IAPNP'", "'TNat'", "'TVacLac'", "'T_Descarte'", "'TPrenhez'", "'T_Cres'", "'COE_RL'", "'LucroLiq_RB'", "'RLO_RB'", "'N_Bezdes_Area'", "'TL_AreaTot'", "'TL_PN'", "'VPUP_Alt'", "'VPUP_Nat'", "'Ano'", "'Area'", "'AreaInund'", "'UA_total_ponderado'", "'RBGado'", "'RL'", "'CT'", "'COE'", "'COT'", "'Lucro_Bruto'", "'CAni_arroba'", "'CDiver_arroba'", "'CFixo_arroba'", "'CFixo_julho2019'", "'CustoDiversos'", "'CustoManejo'", "'Id_PriCria'", "'ProporPN'", "'N_Bezdes'", "'N_Vacas'", "'AreaPInf'", "'AreaPN_US'", "'Valor_PN'", "'AreaPCPN'", "'AreaPN'", "'TDfrute_TDc'", "'T_Desfrute'", "'Prod'", "'TL_PI'", "'TL_US'", "'CFixoCabporM'", "'PC_Nece'", "'Perc_Vacas'", "'nascido'", "'NumCab'", "'PerceRetNov'", "'Nome'", "'SubReg'", "'Perfil'", "'Sist'"])


In [51]:
np_topology_dist_norm = np_topology_dist/np_topology_dist.max()

np_topology_dist_norm = np_topology_dist_norm.round(decimals=2)


In [52]:
import pandas as pd
print(labels)
df_topo_dist_norm = pd.DataFrame(np_topology_dist_norm, columns=labels, index=labels)
print(objectives)

dict_keys(["'PastCult'", "'ICDP'", "'PropAF'", "'PropPInf'", "'PropPN_US'", "'IEP'", "'PC_At'", "'PCPN_AT'", "'GrauInund'", "'IAPNP'", "'TNat'", "'TVacLac'", "'T_Descarte'", "'TPrenhez'", "'T_Cres'", "'COE_RL'", "'LucroLiq_RB'", "'RLO_RB'", "'N_Bezdes_Area'", "'TL_AreaTot'", "'TL_PN'", "'VPUP_Alt'", "'VPUP_Nat'", "'Ano'", "'Area'", "'AreaInund'", "'UA_total_ponderado'", "'RBGado'", "'RL'", "'CT'", "'COE'", "'COT'", "'Lucro_Bruto'", "'CAni_arroba'", "'CDiver_arroba'", "'CFixo_arroba'", "'CFixo_julho2019'", "'CustoDiversos'", "'CustoManejo'", "'Id_PriCria'", "'ProporPN'", "'N_Bezdes'", "'N_Vacas'", "'AreaPInf'", "'AreaPN_US'", "'Valor_PN'", "'AreaPCPN'", "'AreaPN'", "'TDfrute_TDc'", "'T_Desfrute'", "'Prod'", "'TL_PI'", "'TL_US'", "'CFixoCabporM'", "'PC_Nece'", "'Perc_Vacas'", "'nascido'", "'NumCab'", "'PerceRetNov'", "'Nome'", "'SubReg'", "'Perfil'", "'Sist'"])
["'ICDP'"]


In [53]:
df_topo_dist_norm2objectives = df_topo_dist_norm[objectives]

In [54]:
df_topo_dist_norm2objectives = df_topo_dist_norm2objectives.drop(objectives)

In [55]:
df_topo_dist_norm2objectives['Average'] = df_topo_dist_norm2objectives[df_topo_dist_norm2objectives.columns].mean(axis=1) 

In [56]:
df_topo_dist_norm2objectives = ((df_topo_dist_norm2objectives*100)//1)/100

In [57]:
df_topo_dist_norm2objectives.sort_values(by='Average', ascending=True, inplace=True)

#### Evolutionary-time distances from a tree 
- also called cophenetic
 - the evolutionary-time of each branch is used to construct a matrix|

In [58]:
evtime_m, labels = mTree[0].cophenetic_matrix()

In [59]:
evtime_dist = dict()
for l, m in zip(labels, evtime_m):
  evtime_dist[l] = m

#### Cophenetic between clades

Cophenetic between clades

In [60]:
# Choose between a topoloy or evolutionary-time dictionary and creates the corresponding matrix 
def dict2matrix(dist=evtime_dist):#dist=topology_dist
  ml = []
  for l, r in dist.items():
    ml += [r]
  return np.array(ml)
#m = dict2matrix(evtime_dist)


In [61]:
mevo = dict2matrix(evtime_dist)

In [62]:
# Essa célula do notebook é só para mostrar como é o formato dos dados no idsx.
# Com base no formato, você mesmo pode construir o idsx a partir dos rótulos dos nós folhas da árvore.
# Por outro lado, o código da célula acaba sendo uma forma de obter a idsx, 
# pois o idsx é o mesmo para qualquer entrada de NCD que tenha sido obtida para qualquer dados 
# que tenham os mesmos rótulos de folhas. 

path = os.path.join(results,'Plan10') # leaf ids (or names) are the same for all the resampled trees - use Sheet0 or anyother

import ncd_base
import ncd2 as ncd

ncds = [ i for i in os.listdir(path) if '-ncd.csv' in i]
ncdfile = ncds[-1] #the newest one ncd distance matrix in ws.title results
infile = open(os.path.join(path,ncdfile))
generator = ncd_base.csv_read(infile)
ncd_results = list(generator)
m, ids = ncd.to_matrix(ncd_results=ncd_results, is_self=True)
idsx, idsy = ids #len(idsx), idsx 

In [63]:
#idsx, idsy, ids

In [64]:
import clustering
from tree import to_graph
import tree_simplification as nj

In [65]:
tree = nj.neighbor_joining(mevo, ids=idsx) # ids=None)

#####clustering from damicore

In [66]:
g = to_graph(tree)

membership, clustering, dendrogram = clustering.tree_clustering(g, idsx, community_detection_name='fast', is_normalize_weights=True, num_clusters=None)

In [67]:
#print(clustering, dendrogram)
#print (tree)

##### Other membership structures

In [68]:
reverse_membership = dict()
for l, clade in membership.items():
  try:
    reverse_membership[clade] += [l] 
  except:
    reverse_membership[clade]  = [l] 

In [69]:
#membership, reverse_membership

In [70]:
from ete3 import Tree

In [71]:
li = [ (l, i) for i, l in enumerate(Tree(the_tree.newick).get_leaves())]
rev_labels = dict()
for l, i in li:
  rev_labels[l.name.replace("'","")] = i

##### The Cophenetic matrix itself

In [72]:
def clades2cophenetic(m=m, reverse_membership=reverse_membership, rev_labels=rev_labels):
  from itertools import combinations, product
  cophenetic_clades = dict()
  for i1, i2 in combinations(reverse_membership.items(), 2):
    c1, c2 = i1[1], i2[1] #their values, their lists
    c1_c2_pairs = list(product(c1,c2))
    maxpair = 0.0
    for l1, l2 in c1_c2_pairs:
      c1_c2_dist = m[rev_labels[l1]][rev_labels[l2]] 
      if c1_c2_dist > maxpair:
        maxpair = c1_c2_dist
      cophenetic_clades[str(i1[0])+","+str(i2[0])] = (maxpair, l1, l2)
      cophenetic_clades[str(i2[0])+","+str(i1[0])] = (maxpair, l2, l1)
  return cophenetic_clades
#cophenetic_clades = clades2cophenetic(m=m, reverse_membership=reverse_membership, rev_labels=rev_labels)

##### Usage of obtained clade-based cophenetic information

In [73]:
mtop = dict2matrix(topology_dist)
mevo = dict2matrix(evtime_dist)
cophenetic_clades_top = clades2cophenetic(m=mtop, reverse_membership=reverse_membership, rev_labels=rev_labels)
cophenetic_clades_evo = clades2cophenetic(m=mevo, reverse_membership=reverse_membership, rev_labels=rev_labels)

# <font color=green>The tree with clades highlighted</font>
> This is the tree with tips colored by clusters


*   If resampling is active, this is the consensus tree
*   If resambling is not active, this is the original tree

> See configuration varuable `sample_total`



In [74]:
tre = toytree.tree(ctre.newick)

memb_quot = dict()
for m, v in membership.items():
  memb_quot["'"+m+"'"] = v

tre_membership = tre.set_node_values(feature="clade", values=memb_quot)

#colorlist = [ toytree.colors[membership[tip.replace("'",'')]] for tip in tre_membership.get_tip_labels() ]

colorlist = [ alt_colors[membership[tip.replace("'",'')]] for tip in tre_membership.get_tip_labels() ]

# This is the order we naturally obtain for NCD-normalized
custom_order= vars+objectives
# custom_order = [
#  "'L2'",
#  "'L1'",
#  "'L3'",
#  "'D1'",
#  "'Dext'",
#  "'Tfeed'",
#  "'Lfeed'",
#  "'Tbarrel'",
#  "'ERROR'",
#  "'Grooves'",
#  "'RBS'",
#  "'D3'",
#  "'Pitch'",
#  "'N'",
#  "'Screw'",
#  "'ViscousD'",
#  "'Tmelt'",
#  "'WATS'",
#  "'Lmelting'",
#  "'Output'",
#  "'Power'"]


# This is the order we obtain for NCD-non-normalized, but with the two
# major subtrees swapped (one from D1 to Output, the other from Screw to Pitch.)
custom_order_swap= custom_order
# custom_order_swap = [
#  "'L2'",
#  "'L1'",
#  "'L3'",
#  "'Lfeed'",
#  "'N'",

#  "'Screw'",
#  "'ERROR'",
#  "'RBS'",
# "'Grooves'",
#  "'Tfeed'",
#  "'Tbarrel'",
#  "'D3'",
#  "'Dext'",
#  "'Pitch'",
 
#  "'D1'",
#  "'Power'",
#  "'ViscousD'",
#  "'Tmelt'",
#  "'WATS'",
#  "'Lmelting'",
#  "'Output'",
#  ]

# We need to reverse it so that toytree.draw understand it the way we want.

custom_order.reverse()
custom_order_swap.reverse()

tre._fixed_idx = list(range(tre.ntips))  # This is needed.


In [75]:
## draw with node supports and some styling


tree_style = {
    "width":800,
    "height":730,
    #"layout" : 'l',               # Uncomment to reverse direction
    #"node_labels":'support',
    "tip_labels_align":True,
    "tip_labels":True,
    "tip_labels_colors":colorlist,#(['goldenrod'] * 10) + (["mediumseagreen"] * 10),#toytree.colors[1],
    #"node_sizes":25,
    "node_colors":'lightgray',
    "tip_labels_style":{"font-size":"7pt", "font-weight":"normal"},
    "node_labels_style":{"font-size":"4pt", "font-weight":"normal"},
}



from IPython.core.display import display, HTML


if (sample_total > 1):
  display(HTML("<b style='color:blue'>This is the consensus tree</b>"))
  node_labels_ = "support"
  node_sizes_ = 10
  #canvas, axes, mark = tre.draw(**tree_style, node_labels="support", node_sizes=25, shrink=10)
else:
  display(HTML("<b style='color:blue'>This is the original tree</b>"))
  node_labels_ = False
  node_sizes_ = 10
  #canvas, axes, mark = tre.draw(**tree_style, node_labels=False, node_sizes=10, shrink=10)

print('Selected tree:        ', 'consensus of '+str(sample_total) if sample_total > 1 else 0)
print('Input normalization:  ', 'yes' if (transf_normalize==True) else "no")
print('Input delta-filter:   ', 'yes' if (transf_delta==True) else "no")
print('Distance metric:      ', dist_type)
for i in range(len(columns)):
  print(str(i),':',columns[i])

#canvas, axes, mark = tre.draw(**tree_style, fixed_order=custom_order, node_labels=node_labels_, node_sizes=node_sizes_, shrink=10)
canvas, axes, mark = tre.draw(**tree_style, node_labels=node_labels_, node_sizes=node_sizes_, shrink=10)

Selected tree:         consensus of 32
Input normalization:   no
Input delta-filter:    no
Distance metric:       ncd
0 : Nome_fazenda
1 : Sub_regiao
2 : Ano
3 : Sistema
4 : Tamanho_ha
5 : Perfil_de_gestao
6 : Area_pastagem_cultivada_ha
7 : Proporcao_de_formacao_de_pastagem_cultivada_da_area_total
8 : Proporcao_de_pastagem_nativa_exotica_da_area
9 : area_pastagem_nativa_totalexotica
10 : area_de_pastagem_nativa_total_ha
11 : Proporcao_de_pastagem_nativa_total_
12 : Grau_de_inundacao_cheia_
13 : area_iundada_ha
14 : Proporcao_pasto_nativo_de_melhor_qualidade_area_umida_e_sazonal
15 : Area_pastagem_nativa_melhor_qualidade_ha
16 : Taxa_de_lotacao_media_UAha
17 : Proporcao_de_areas_florestadas_natural__da_area_total
18 : Proporcao_de_pastagens_de_qualidade_inferior_savanas_e_campos_sujos_altos_da_area_total
19 : Area_pastagem__qualidade_inferior
20 : Taxa_de_lotacao_media_
21 : Valorprodutivo_paisagens__VPUP_natural_relacao
22 : Valorprodutivo_paisagens__VPUP_alteradonatural_relacao
23 : i

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="730.0px" viewBox="0 0 800.0 730.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc1fd2bbf6e6d431f8c39987bce07f27c"> 72 66 72 69 44 50 88 78 94 78 25 50 59 44 47 38 50 53 100 75 53 88 75 100 62 62 28 16 12 72 38 19 44 78 78 22 12 3 53 91 100 84 9 16 41 41 22 19 47 66 3 22 94 100 12 31 56 100 6 53 100 'CDiver_arroba' 'CFixo_arroba' 'CAni_arroba' 'CustoManejo' 'CustoDiversos' 'CFixo_julho2019' 'Lucro_Bruto' 'COE' 'COT' 'CT' 'RBGado' 'RL' 'UA_total_ponderado' 'AreaInund' 'Area' 'PastCult' 'TNat' 'TVacLac' 'ICDP' 'T_Descarte' 'T_Cres' 'TPrenhez' 'PropPInf' 'PropPN_US' 'PropAF' 'IEP' 'IAPNP' 'GrauInund' 'PCPN_AT' 'PC_At' 'N_Bezdes_Area' 'TL_AreaTot' 'VPUP_Alt' 'VPUP_Nat' 'RLO_RB' 'LucroLiq_RB' 'COE_RL' 'AreaPCPN' 'AreaPN' 'AreaPInf' 'Perc_Vacas' 'PC_Nece' 'CFixoCabporM' 'TL_PI' 'TL_US' 'AreaPN_US' 'Prod' 'T_Desfrute' 'TDfrute_TDc' 'SubReg' 'Nome' 'Sist' 'Perfil' 'PerceRetNov' 'NumCab' 'nascido' 'TL_PN' 'Valor_PN' 'N_Vacas' 'N_Bezdes' 'ProporPN' 'Id_PriCria' 'Ano'

# <font color=blue>Save results</font>

## Save raw output

In [76]:
!pip install ghostscript
!apt install ghostscript python3-tk

import ghostscript
import toyplot.png

save_path = "/content/output"

os.makedirs(save_path, exist_ok=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.8.10-0ubuntu1~20.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common
  libidn11 libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9
  libgs9-common libidn11 libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 23 not upgraded.
Need to get 12.7 MB of archives.
After this operati

In [77]:
# Save the colored tree



toyplot.png.render(canvas, os.path.join(save_path,"colortree.png"))

# Save the tree's newick file

file = open(os.path.join(save_path,"colortree_newick.txt"), "w")
file.write(the_tree.newick)
file.close()


# Save reverse membership (Table 2)

file = open(os.path.join(save_path,"reverse-membership"), "w")
file.write(str(reverse_membership))
file.close()

# Save cophenetic_clades_top (Table 3)

file = open(os.path.join(save_path,"cophenetic_clades_top"), "w")
file.write(str(cophenetic_clades_top))
file.close()

# Save df_topo_dist_norm2objectives (Table 4)

df_topo_dist_norm2objectives.to_csv(os.path.join(save_path,"np_topology_dist_norm2objectives.csv"), sep=',', mode='w', decimal=".")



## Save docx file

In [78]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=15398e832dd2d058dbb18d9b4988ceea79e6b4bdbbf236e8efe0f56d812df2d2
  Stored in directory: /root/.cache/pip/wheels/83/8b/7c/09ae60c42c7ba4ed2dddaf2b8b9186cb105255856d6ed3dba5
Successfully built python-docx


In [79]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH


doc = Document()

doc.add_heading('Output report', level=0)

p = doc.add_paragraph('Output from last execution')
p = doc.add_paragraph()
run = p.add_run('Input data: ').bold = True
p = doc.add_paragraph()
run = p.add_run(data_path)
run.font.name = 'Courier New'
run.font.size=Pt(10)


print('Selected tree:        ', 'consensus of '+str(sample_total) if sample_total > 1 else 0)
print('Input normalization:  ', 'yes' if (transf_normalize==True) else "no")
print('Input delta-filter:   ', 'yes' if (transf_delta==True) else "no")
print('Distance metric:      ', dist_type)

#p = doc.add_paragraph()

p = doc.add_paragraph('Selected tree:        ' + (('consensus of ' + str(sample_total)) if sample_total > 1 else str(0)), style="List Bullet")
p = doc.add_paragraph('Input normalization:  ' + ('yes' if (transf_normalize==True) else "no"), style="List Bullet")
p = doc.add_paragraph('Input delta-filter:   ' + ('yes' if (transf_delta==True) else "no"), style="List Bullet")
p = doc.add_paragraph('Distance metric:      ' + dist_type, style="List Bullet")

##
## Tree view
##

doc.add_heading('Tree visualization')

doc.add_picture(os.path.join(save_path,"colortree.png"))
#doc.add_page_break()

#color_names = ['Cyan', 'Orange','Blue', 'Magenta', 'Green', 'Yellow', 'Brown', 'Gray']


##
## Level-one learning
##

doc.add_heading('Level-one learning')

doc.add_paragraph()
table = doc.add_table(len(reverse_membership)+1,2)
table.style = 'Table Grid'

run = table.cell(0,0).paragraphs[0].add_run('Clade')
run.bold = True
run = table.cell(0,1).paragraphs[0].add_run('Variables sharing information')
run.bold = True

for i in reverse_membership:
  #print(i, reverse_membership[i])
  row = table.rows[i+1]
  row.cells[0].text = str(i) + ' - ' + color_names[i]
  for j in reverse_membership[i]:
    if not "'"  +j + "'" in objectives:
      row.cells[1].text += j + " "
  if len(row.cells[1].text) == 0:
    row.cells[1].text = '(only objectives)'  





Selected tree:         consensus of 32
Input normalization:   no
Input delta-filter:    no
Distance metric:       ncd


In [80]:

##
## Level-two learning: vclates to oclades
##

doc.add_heading('Level-two learning: vclades to oclades') 

# Build a dataframe from cophenetic_clades_top dictionary

aux_df = pd.DataFrame()
for i, v in cophenetic_clades_top.items():
  label1, label2 = i.split(',')
  dist = v[0]
  tip1 = v[1]
  tip2 = v[2]
  aux_df = aux_df.append({'Label1':label1,
                    'Label2':label2,
                    'Dist':dist,
                    'Tip1':tip1,
                    'Tip2':tip2}, ignore_index=True)
    
aux_df.sort_values(by='Label1',inplace=True)

# Clean up excess quotes from objectives and vars

obj = {i.replace("'","") for i in objectives} 
vr = {i.replace("'","") for i in vars} 

# We need two lists
#    one of clades that contains objectis  (these will be our colums)
#    one of clades that contains variables (these will be our rows)

oclade_cols = []
for i,v in reverse_membership.items():
  if set(v) & obj:
    oclade_cols += [i]

vclade_rows = []
for i,v in reverse_membership.items():
  if set(v) & vr:
    vclade_rows += [i]



<ipython-input-80-465c5df0dd17>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aux_df = aux_df.append({'Label1':label1,
<ipython-input-80-465c5df0dd17>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aux_df = aux_df.append({'Label1':label1,
<ipython-input-80-465c5df0dd17>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aux_df = aux_df.append({'Label1':label1,
<ipython-input-80-465c5df0dd17>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aux_df = aux_df.append({'Label1':label1,
<ipython-input-80-465c5df0dd17>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [81]:

# Let's build our output dataframe

df4 = pd.DataFrame()

for i, row in aux_df.iterrows():
  label1 = row['Label1']
  label2 = row['Label2']
  dist = row['Dist']
  if (label1 in str(vclade_rows)) and (label2 in str(oclade_cols)):
    #print (label1, label2, 'dist', dist)
    df4.loc[label1,label2] = dist
  


In [82]:

# Format docx table

doc.add_paragraph()

tab4 = doc.add_table(df4.shape[0]+2,df4.shape[1]+1)
tab4.style = 'Table Grid'

# Add index
for i in range(df4.shape[0]):
  tab4.cell(i+2,0).text = df4.index[i] + ' - ' + color_names[int(df4.index[i])]

run = tab4.cell(0,0).paragraphs[0].add_run(' ')
run = tab4.cell(0,1).paragraphs[0].add_run('node in oclade')
run.bold=True

run = tab4.cell(0,1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

run = tab4.cell(1,0).paragraphs[0].add_run('node in vclades')
run.bold=True



#add the header rows.
for j in range(df4.shape[-1]):
  run = tab4.cell(1,j+1).paragraphs[0].add_run(df4.columns[j] + ' - ' + color_names[int(df4.columns[j])])
  run.bold=True


#add the rest of the data frame
for i in range(df4.shape[0]):
  for j in range(df4.shape[-1]):
    #tab4.cell(i+2,j+1).text = str(df4.values[i,j])
    if not df4.values[i,j] == df4.values[i,j]:
      tab4.cell(i+2,j+1).text = ' - '
    else:
      tab4.cell(i+2,j+1).text = str(df4.values[i,j])

first = tab4.cell(0,1)
last = tab4.cell(0,df4.shape[1])
first.merge(last) 


In [83]:
##
## Level-two learning: var to objectis
##

# Move ERROR to the bottom row (for visualization)
# idx = df_topo_dist_norm2objectives.index.tolist()
# the_row = df_topo_dist_norm2objectives.loc["'ERROR'"] 
# the_idx = idx.index("'ERROR'")
# idx.pop(the_idx)
# df_topo_dist_norm2objectives = df_topo_dist_norm2objectives.reindex(idx+["'ERROR'"])

df = df_topo_dist_norm2objectives

doc.add_page_break()

doc.add_heading('Level-two learning: var to objective')
doc.add_paragraph()

t = doc.add_table(df.shape[0]+2, df.shape[1]+1)
t.style = 'Table Grid'

print(t)

for i in range(df.shape[0]):
  t.cell(i+2,0).text = str(df.index[i])#.strip("'")

run = t.cell(0,0).paragraphs[0].add_run(' ')
run = t.cell(0,1).paragraphs[0].add_run('Objectives')
run.bold = True


t.cell(0,1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

run = t.cell(1,0).paragraphs[0].add_run('Variables')
run.bold = True

# add the header rows.
for j in range(df.shape[-1]):
    #t.cell(0,j+1).text = df.columns[j]
    run = t.cell(1,j+1).paragraphs[0].add_run(df.columns[j].strip("'"))
    run.bold=True

# add the rest of the data frame
for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        t.cell(i+2,j+1).text = str(df.values[i,j])
        
first = t.cell(0,1)
last = t.cell(0, j)
first.merge(last)

In [84]:
from google.colab import drive
from google.colab import files
import os


# Handy function to report errors.

def fault(msg):
  raise ValueError(msg)

# Mount the google drive

#drive.mount('/gdrive/Shareddrives/Data_Embrapa/Planilhas')
drive.mount('/gdrive/')
doc.save(report_path)

Mounted at /gdrive/


# Notes


## Todo

**Removed unused code**

**Change mtree policy**

> Currently, if we want to draw one unique tree we need to set `samples_total=1`, and have the original tree. It would be beteter if we could have many samples and then choose to show one or the consensus tree.

**Fix excess quotes from labels**

**Easy configuration of fixed order**

